---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32

In [6]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import recall_score, accuracy_score, precision_score
from sklearn.metrics import roc_curve, auc
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor

scaler = MinMaxScaler()
train = pd.read_csv("train.csv", encoding="ISO-8859-1")
test = pd.read_csv("test.csv", encoding="ISO-8859-1")


for c in test.columns:
    if len(test[c].unique()) < 10 and len(test[c].unique()) > 2:
        i = 0

        uni_values = list(set(test[c].unique()) | set(train[c].unique()))

        for label in uni_values:
            test[c][test[c] == label] = np.float64(i)
            train[c][train[c] == label] = np.float64(i)
            i += 1

        test[c] = test[c].astype(np.float64)
        train[c] = train[c].astype(np.float64)


train = train.set_index(["ticket_id"])
test = test.set_index(["ticket_id"])
test = test.drop("non_us_str_code", axis=1)

test = test.select_dtypes(include=[np.float64])

for col in test.columns:
    test[col].replace(to_replace=np.NaN, value=np.nanmedian(np.float64(test[col])))
    test[col] = np.nan_to_num(test[col])

compliance = train["compliance"]
train = train[test.columns]
train["compliance"] = compliance
train = train.select_dtypes(include=[np.float64])
train = train[(train["compliance"] == 1) | (train["compliance"] == 0)]

for col in train.columns:
    train[col].replace(to_replace=np.NaN, value=np.nanmedian(np.float64(train[col])))
    train[col] = np.nan_to_num(train[col])

X = train.iloc[:, :-1]
y = train.iloc[:, -1]


X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
test_scaled = scaler.transform(test)


def print_stats(y_test, y_prediction):
    print("accuracy  = {}".format(accuracy_score(y_test, y_prediction)))
    print("recall    = {}".format(recall_score(y_test, y_prediction)))
    print("precision = {}".format(precision_score(y_test, y_prediction)))
    fpr_lr, tpr_lr, _ = roc_curve(y_test, y_prediction)
    roc_auc_lr = auc(fpr_lr, tpr_lr)
    print("auc       = {}\n".format(roc_auc_lr))


def blight_model():
    model = RandomForestRegressor()
    model.fit(X_train, y_train)
    y_prediction = model.predict(X_test)

    y_prediction_abs = np.float64(y_prediction >= 0.5)

#     print_stats(y_test, y_prediction_abs)

    return pd.Series(data=model.predict(test), index=test.index, name="compliance")


blight_model()


/opt/conda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


accuracy  = 0.9230172629472104
recall    = 0.3482019363762102
precision = 0.4579354251932697
auc       = 0.6580267462775382



ticket_id
284932    0.000000
285362    0.000000
285361    0.000000
285338    0.000000
285346    0.150000
285345    0.000000
285347    0.000000
285342    1.000000
285530    0.000000
284989    0.000000
285344    0.000000
285343    0.000000
285340    0.100000
285341    0.000000
285349    0.000000
285348    0.000000
284991    0.000000
285532    0.000000
285406    0.000000
285001    0.000000
285006    0.000000
285405    0.000000
285337    0.000000
285496    0.000000
285497    0.100000
285378    0.000000
285589    0.000000
285585    0.000000
285501    0.000000
285581    0.000000
            ...   
376367    0.000000
376366    0.000000
376362    0.000000
376363    0.300000
376365    0.000000
376364    0.000000
376228    0.203452
376265    0.000000
376286    0.300000
376320    0.000000
376314    0.000000
376327    0.900000
376385    0.000000
376435    1.000000
376370    1.000000
376434    0.000000
376459    0.000000
376478    0.000000
376473    0.000000
376484    0.000000
376482    0.000000
37